In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np

# 데이터 로드
data = pd.read_csv('data.csv')

# 데이터 탐색
print("데이터 샘플:\n", data.head())
print("\n데이터 정보:\n")
data.info()
print("\n기술 통계:\n", data.describe())

# 결측치 확인
print("\n결측치 개수:\n", data.isnull().sum())

# 결측치 처리
from sklearn.impute import SimpleImputer

# 수치형 변수와 범주형 변수 구분
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()

# 수치형 변수 결측치 처리 (평균 대체)
mean_imputer = SimpleImputer(strategy='mean')
data[numerical_cols] = mean_imputer.fit_transform(data[numerical_cols])

# 범주형 변수 결측치 처리 (최빈값 대체)
mode_imputer = SimpleImputer(strategy='most_frequent')
data[categorical_cols] = mode_imputer.fit_transform(data[categorical_cols])

# 범주형 변수 인코딩 (원-핫 인코딩)
data = pd.get_dummies(data, columns=categorical_cols)

# 특징과 레이블 분리
X = data.drop('target', axis=1)
y = data['target']

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 데이터 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 레이블 일부 제거
import random

random_state = np.random.RandomState(42)
n_samples = len(y_encoded)
n_labels = int(n_samples * 0.5)  # 50%만 레이블 유지
unlabeled_indices = random_state.choice(n_samples, n_samples - n_labels, replace=False)
y_semi = np.copy(y_encoded)
y_semi[unlabeled_indices] = -1  # 레이블 없는 데이터는 -1로 설정

# 준지도 학습 모델 정의
from sklearn.semi_supervised import LabelSpreading

model = LabelSpreading(kernel='rbf', gamma=0.25, max_iter=1000)

# 모델 학습
model.fit(X_scaled, y_semi)

# 예측 수행
y_pred = model.transduction_

# 레이블 복원
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_true_labels = label_encoder.inverse_transform(y_encoded)

# 모델 평가
from sklearn.metrics import accuracy_score, classification_report

print("\n전체 데이터에 대한 정확도:", accuracy_score(y_true_labels, y_pred_labels))
print("\n분류 보고서:\n", classification_report(y_true_labels, y_pred_labels))

# 레이블이 없는 데이터에 대한 정확도
print("\n레이블이 없는 데이터에 대한 정확도:", accuracy_score(
    y_true_labels[unlabeled_indices], y_pred_labels[unlabeled_indices]))